In [1]:
import numpy as np
import sympy as sp
import itertools as it
import string
import pickle

In [2]:
#definizioni
part_diz = { 'A' : 3, 'B' : 2}
ind_part = []
parties = list(part_diz.keys())
inputs = [part_diz[i] for i in parties]
max_in=[str(i) for i in range(max(inputs))]
operators=[]
for i in list(part_diz.keys()):
    tmp=[]
    for l in range(part_diz[i]):
        tmp.append(str(l))
        print(str(l))
    operators+=[''.join(k) for k in it.product(i, tmp, repeat = 1)]

level = 1

0
1
2
0
1


In [3]:
blabla=[[i for i in range(j)] for j in inputs]

In [4]:
blabla

[[0, 1, 2], [0, 1]]

In [5]:
filename = f'indici_dei_constraints_instrumental_lev_{level}'
with open(filename, 'rb') as g:
    index_keys = pickle.load(g)
filename = f'matrici_dei_constraints_instrumental_lev_{level}'
with open(filename, 'rb') as f:
    dizionario_cons = pickle.load(f)
diz = { index_keys[i] : dizionario_cons[i]  for i in range(len(index_keys)) }
diz['']=diz['']/2.

In [6]:
n=2
independent_monomials = []
independent_monomials_highlight = {}
index_indep_keys=[]
for i in range(len(index_keys)):
    char=[]
    for x in ''.join(sorted(index_keys[i])):
        if x.isalpha():
            char.append(x)
    char = ''.join(sorted(list(set(char))))
    for a in ind_part:
        if a in char:
            independent_monomials.append(index_keys[i])
            #print(index_keys[i])
            lista = [index_keys[i][j:j+n] for j in range(0, len(index_keys[i]), n) ]
            #print(lista)
            index_indep_keys=lista[0]
            for k in range(len(lista)-1):
                if(lista[k][0]+lista[k+1][0] not in ind_part):
                    index_indep_keys = ''.join([index_indep_keys, lista[k+1]])
                else:
                    index_indep_keys = '*'.join([index_indep_keys, lista[k+1]])
            independent_monomials_highlight[index_indep_keys]=diz[index_keys[i]]

In [7]:
non_mes_diz={}
mes_diz={}
for i in range(len(diz.keys())):
    if any(l+l in ''.join(sorted(index_keys[i])) for l in parties):
        non_mes_diz[index_keys[i]]=diz[index_keys[i]]
    else:
        mes_diz[index_keys[i]]=diz[index_keys[i]]

In [8]:
filename = f'matrici_mes_constraints_instrumental_lev_{level}'
with open(filename, 'wb') as f:
    pickle.dump(mes_diz, f)
filename = f'matrici_non_mes_constraints_instrumental_lev_{level}'
with open(filename, 'wb') as f:
    pickle.dump(non_mes_diz, f)

In [9]:
def expected_value(p, operator, parties, inputs):
    n=2
    suffix_operators={}
    suffix=0
    e_value=0
    inputs_voc={}
    for i in range(len(inputs)):
        inputs_voc[inputs[i]]=i 
    for i in range(len(parties)):
        suffix_operators[parties[len(parties)-1-i]]=i
    list_operators = [operator[j:j+1] for j in range(0, len(operator), n) ]
    list_inputs = [operator[j+1:j+2] for j in range(0, len(operator), n) ]
    if(len(inputs)>1):
        for i in range(len(list_operators)):
            suffix+=2**(len(parties)+suffix_operators[list_operators[i]])*inputs_voc[list_inputs[i]]
        #print(suffix)
    for i in range(2**len(parties)):
        binar=bin(i+2**(len(parties)))[3:]
        coeff = 0
        print(binar)
        for j in list_operators:
            coeff+=int(binar[len(parties)-1-suffix_operators[j]])
        e_value+=(-1)**(coeff)*p[suffix+i]
    return e_value
inputs = ['0','1','2']
parties = ['A','B']
P_pred=np.array([0.52318174, 0.06456647, 0.08200467, 0.33024713,
                 0.45038733, 0.07953553, 0.06805175, 0.4020253 ,
                 0.5017574 , 0.06083163,  0.08692036, 0.35049063,
                 0.07752251, 0.4612264 , 0.39155433,  0.06969678,
                 0.4697568 , 0.53024316, 0, 0,
                 0.52331525, 0.47668475, 0, 0])

In [10]:
def chsh(x):
    vals=[0,1]
    a=sum((-1)**(a+b)*x[a*2+b] for a in vals for b in vals)
    b=sum((-1)**(a+b)*x[a*2+b+4] for a in vals for b in vals)
    c=sum((-1)**(a+b)*x[a*2+b+8] for a in vals for b in vals)
    d=sum((-1)**(a+b)*x[a*2+b+12] for a in vals for b in vals)
    return a+b+c-d

In [11]:
val=(expected_value(P_pred, 'A0B0', parties, inputs)+
expected_value(P_pred, 'A0B1', parties, inputs)+
expected_value(P_pred, 'A1B0', parties, inputs)-
expected_value(P_pred, 'A1B1', parties, inputs))
print(val, chsh(P_pred), expected_value(P_pred, 'A2B1', parties, inputs))

00
01
10
11
00
01
10
11
00
01
10
11
00
01
10
11
00
01
10
11
2.82174056 2.82174056 0.04663050000000002


In [12]:
p=np.array([0.07488913, 0.4285545, 0.42794433, 0.06861203, 0.06410121, 0.42656192, 
            0.42814913, 0.08118775, 0.07411963, 0.41813782, 0.43292814, 0.07481445,
            0.59310925, 0.06043877, 0.09045572, 0.25599623])
P_=np.array([0.0732233, 0.4267767, 0.4267767, 0.0732233, 0.0732233, 0.4267767,
             0.4267767, 0.0732233, 0.0732233, 0.4267767, 0.4267767, 0.0732233, 
             0.5267767, 0.0732233, 0.0732233, 0.3267767])
chsh(P_)

-2.8284272

In [13]:
vals=[0,1]
prod=0
for b in vals:
    for y in vals:
        prod+=abs(sum(p[a*2+b+y*4]-p[a*2+b+8+y*4] for a in vals))
prod

0.3910578700000001

In [14]:
print(sum(abs(P_[i]-p[i]) for i in range(16))/8.)
print(chsh(p), chsh(P_))

0.026538006250000003
-2.8227626299999997 -2.8284272


In [15]:
P_

array([0.0732233, 0.4267767, 0.4267767, 0.0732233, 0.0732233, 0.4267767,
       0.4267767, 0.0732233, 0.0732233, 0.4267767, 0.4267767, 0.0732233,
       0.5267767, 0.0732233, 0.0732233, 0.3267767])

In [16]:
non_mes_diz

{'A1A0': array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]),
 'A2A0': array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]),
 'A2A1': array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]),
 'B1B0': array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.]])}